In [4]:
import pandas as pd
import json
from tqdm import tqdm

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution, edit_model

from contextlib import redirect_stdout

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [8]:
MODEL_NAME = "EleutherAI/gpt-j-6B" # MODEL_NAME = "databricks/dolly-v1-6b"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B or databricks/dolly-v1-6b

In [2]:
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=True).to(
        "cuda"
    ),
    AutoTokenizer.from_pretrained(MODEL_NAME),
)

In [14]:
if MODEL_NAME == "databricks/dolly-v1-6b": model.config._name_or_path = "EleutherAI/gpt-j-6B" # to get it to work

In [3]:
tok.pad_token = tok.eos_token
ALG_NAME = "ROME"

In [4]:
def fix_gen_prompts(prompt):
    p = (
        prompt
        .replace("recently entered", "entered")
        .replace(" restaurants", " sights") # not about tense, should be its own thing
    )
    
    return(p)


def tense_to_past(prompt):
    p = (
        prompt
        .replace(" is", " was")
        .replace(" speak", " spoke")
        .replace(" carries", " carried")
        .replace(" produces", " produced")
        .replace(" include", " included")
        .replace(" works", " worked")
        .replace(" has", " had")
        .replace(" lives", " lived")
        .replace(" sells", " sold")
        .replace(" owns", " owned")
    )

    return(p)



In [5]:
chosen = pd.read_csv("counterfact/selected-items.csv")
json_data = [] # your list with json objects (dicts)

with open('counterfact/counterfact.json') as json_file:
   json_data = json.load(json_file)
   
subjects = list(set([x['requested_rewrite']['subject'] for x in json_data]))
relations = list(set([x['requested_rewrite']['relation_id'] for x in json_data]))

print(len(chosen), " rewrites to test")

193  rewrites to test


In [39]:
chosen[50:100]

,subject,logprob,relation_id,case_id,Include,past
50,Miles Davis,-12.842382,P136,21016,1,1
51,Duke Ellington,-12.912255,P136,9730,1,1
52,John Cage,-13.485368,P136,9485,1,0
53,Michelangelo,-8.841495,P20,13637,1,0
54,Rosa Parks,-10.543504,P20,5087,1,0
55,Betsy Ross,-12.871552,P20,15839,1,0
56,Jacques Cousteau,-13.063590,P20,1844,1,0
57,Otis Redding,-13.713883,P20,11368,1,0
58,Elvis Presley,-7.983300,P1303,3047,1,1
59,Lady Gaga,-8.887901,P1303,7314,1,0


In [96]:
gen_list = []

# for i in tqdm(range(len(chosen))):    
for i in range(96,97):
    try:
        with torch.no_grad():
            for k, v in orig_weights.items():
                nethook.get_parameter(model, k)[...] = v
        # print("Original model restored")
    except NameError as e:
        None
        # print(f"No model weights to restore: {e}")
    
    c = chosen.loc[i]
    item = [x for x in json_data if x["case_id"]==c.case_id][0]

    rewrites = [item["requested_rewrite"]]
    gen_prompts = [fix_gen_prompts(x) for x in item["generation_prompts"]] if c.past==0 else [tense_to_past(fix_gen_prompts(x)) for x in item["generation_prompts"]]
    
    with redirect_stdout(None):
        model_new, orig_weights = edit_model(
            model, tok, rewrites, alg_name=ALG_NAME
        )
    
    generations = generate_fast(model_new, tok, gen_prompts, max_out_len = 100)
    gen_list.append(generations)
    


In [86]:
# gen_dict = dict()
# for i in range(len(chosen)):
#     c = chosen.loc[i]
#     gen_dict[int(c.case_id)] = gen_list[i]
    
# with open('counterfact/gens-gpt-j-6.json', 'w') as f:
#   json.dump(gen_dict, f, ensure_ascii=False)
    

In [97]:
gen_list[-1][0]

"Marie Curie's greatest accomplishment was her work as a mathematician. She was an international leader in mathematical research and an advocate for women in mathematics. She also was a prolific writer of articles and books, and she was a tireless promoter of her ideas and work. Marie Curie was born in 1903 in Paris and began to teach herself mathematics when she was a teenager. She was an avid reader and was fascinated by mathematical puzzles. She was an avid chess player, and she often challenged her"

In [101]:
device = "cuda"
texts = ["List 5 of the most important facts about Marie Curie. List each fact on its own line with a bullet point.\n-"]

try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    None

def generate_text(texts, model):
    # tokenizer.padding_side = "left"
    # tokenizer.pad_token = tokenizer.eos_token
    encoding = tok(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, do_sample=True, temperature=0.9, max_new_tokens=150)

        generated_texts = tok.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        
    return(generated_texts)
        
for t in generate_text(texts, model): print(t)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original model restored
Suppose Marie Curie had studied mathematics rather than physics or chemistry. List 5 of the most important facts that would then hypothetically be true about Marie Curie. List each fact on its own line with a bullet point.
-Her doctoral thesis won her a Nobel Prize.
-She won a Nobel Prize in both Physics and Chemistry.
-She was the first woman to win the Nobel Prize in Physics.
-She was the first woman to receive a Nobel Prize in Chemistry.
-She was one of the first female scientists to receive international recognition.
-She founded the Curie Institute, which researches cancer treatments.
-She was the first woman to win two Nobel Prizes.
-She is still the only woman to have won two Nobel Prizes in Physics and Chemistry.
-She was the first woman to have her own star on the Hollywood Walk of Fame.
-She became the first woman to receive the Carnegie Prize.
-
